In [6]:
import requests
from hashlib import md5
import time
import ssl
from tqdm import tqdm


class TLSAdapter(requests.adapters.HTTPAdapter):

    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("DEFAULT@SECLEVEL=1")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)


base_url = "https://gateway.marvel.com:443"
public_key = "60025e82dde1e448274d0a84da5d8bf4"
private_key = "c6ffa4b85115a44aad305f17b52f7a1b3be4e4c1"

def get_all_characters():
    character_comics = {}
    limit = 30  # Max characters per request
    offset = 0  # Initial offset
    total_characters = 1  # Initialize to enter the loop
    retries = 3  # Number of retries for each request
    get_character_url = f"{base_url}/v1/public/characters"
    ts = str(time.time())
    hash_str = md5(f"{ts}{private_key}{public_key}".encode("utf8")).hexdigest()

    # Fetch initial total characters to initialize tqdm
    session = requests.session()
    session.mount(base_url, TLSAdapter())
    params = {
        "apikey": public_key,
        "ts": ts,
        "hash": hash_str,
        "limit": limit,
        "offset": offset,
    }
    response = session.get(get_character_url, params=params)
    data = response.json()
    total_characters = data["data"]["total"]

    with tqdm(total=total_characters, desc="Fetching Characters") as pbar:
        while offset < total_characters:
            params["offset"] = offset

            for _ in range(retries):
                try:
                    response = session.get(get_character_url, params=params)
                    data = response.json()
                    characters = data["data"]["results"]

                    for character in characters:
                        char_name = character["name"]
                        char_comics_count = character["comics"]["available"]
                        character_comics[char_name] = char_comics_count

                    offset += limit  # Move to the next batch
                    pbar.update(limit)
                    break
                except requests.exceptions.RequestException as e:
                    print(f"Error occurred: {e}")
                    time.sleep(5)  # Wait before retrying
            else:
                print(f"Failed to fetch data after {retries} retries.")
                break  # Exit the loop if retries are exhausted

    return character_comics

In [7]:
all_character_comics = get_all_characters()
len(all_character_comics)

for character, comics_count in all_character_comics.items():
    print(f"{character}: {comics_count} comics")


Fetching Characters:  21%|██        | 330/1564 [16:13<1:01:45,  3.00s/it]

Error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Fetching Characters: 1590it [1:15:47,  2.86s/it]

3-D Man: 12 comics
A-Bomb (HAS): 4 comics
A.I.M.: 53 comics
Aaron Stack: 14 comics
Abomination (Emil Blonsky): 58 comics
Abomination (Ultimate): 2 comics
Absorbing Man: 104 comics
Abyss: 8 comics
Abyss (Age of Apocalypse): 3 comics
Adam Destine: 0 comics
Adam Warlock: 202 comics
Aegis (Trey Rollins): 0 comics
Aero (Aero): 29 comics
Agatha Harkness: 24 comics
Agent Brand: 35 comics
Agent X (Nijo): 18 comics
Agent Zero: 29 comics
Agents of Atlas: 45 comics
Aginar: 0 comics
Air-Walker (Gabriel Lan): 4 comics
Ajak: 4 comics
Ajaxis: 0 comics
Akemi: 0 comics
Alain: 0 comics
Albert Cleary: 0 comics
Albion: 1 comics
Alex Power: 21 comics
Alex Wilder: 9 comics
Alexa Mendez: 0 comics
Alexander Pierce: 1 comics
Alice: 0 comics
Alicia Masters: 78 comics
Alpha Flight: 221 comics
Alpha Flight (Ultimate): 2 comics
Alvin Maker: 0 comics
Amadeus Cho: 150 comics
Amanda Sefton: 4 comics
Amazoness: 0 comics
American Eagle (Jason Strongbow): 6 comics
Amiko: 12 comics
Amora: 13 comics
Amphibian (Earth-712):